# Assignment 2

Based on https://serv.cusp.nyu.edu/~hvo/files/SQL_Lab.pdf

In [1]:
try:
    import urllib2 as ulib
except ImportError:
    import urllib as ulib #used to be urllib3 originally, for compataibility with python 3

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd

In [2]:
API_URL = 'https://fb55.carto.com:443/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = API_URL):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

# Task 1 — Familiarize with SQL Clauses

1.1: Sort data by start_station_id, tripduration

- Only checking trips with duration <= 3 hours

1.2: Only show the top/last 10 records (aka head and tail in SQL)

1.3: List all unique start_station_id values

1.4: Aggregation functions:
- Count the number of trips (aka wc -l in SQL)
- Find the average/min/max trip duration

In [3]:
#TASK 1.1
#Sort data by start_station_id, tripduration
#Only checking trips with duration <= 3 hours

query = '''
SELECT start_station_id, tripduration FROM fb55.citibike
WHERE tripduration <=10800
ORDER BY start_station_id ASC, tripduration ASC;  
'''
table = get_data(query)
table.head(5)

,start_station_id,tripduration
0,72,107
1,72,189
2,72,229
3,72,248
4,72,252


In [4]:
#TASK 1.2

query = '''
SELECT * FROM fb55.citibike
ORDER BY cartodb_id ASC
Limit 10;  
'''
table = get_data(query)
table

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,8 Ave & W 31 St,NaN,70,-73.994811,801,2015-02-01 00:00:00+00,2015-02-01 00:14:00+00,521,1,40.750450,423,W 54 St & 9 Ave,40.765849,-73.986905,17131,Subscriber,1978,2
1,NaN,E 17 St & Broadway,NaN,71,-73.990093,379,2015-02-01 00:00:00+00,2015-02-01 00:07:00+00,497,2,40.737050,504,1 Ave & E 15 St,40.732219,-73.981656,21289,Subscriber,1993,1
2,NaN,Grand Army Plaza & Central Park S,NaN,72,-73.973715,2474,2015-02-01 00:01:00+00,2015-02-01 00:42:00+00,281,3,40.764397,127,Barrow St & Hudson St,40.731724,-74.006744,18903,Subscriber,1969,2
3,NaN,6 Ave & Broome St,NaN,73,-74.004704,818,2015-02-01 00:01:00+00,2015-02-01 00:15:00+00,2004,4,40.724399,505,6 Ave & W 33 St,40.749013,-73.988484,21044,Subscriber,1985,2
4,NaN,Lawrence St & Willoughby St,NaN,74,-73.986317,544,2015-02-01 00:01:00+00,2015-02-01 00:10:00+00,323,5,40.692362,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,19868,Subscriber,1957,1
5,NaN,Willoughby Ave & Walworth St,NaN,75,-73.953820,717,2015-02-01 00:02:00+00,2015-02-01 00:14:00+00,373,6,40.693317,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,15854,Subscriber,1979,1
6,NaN,W 56 St & 6 Ave,NaN,76,-73.977225,1306,2015-02-01 00:04:00+00,2015-02-01 00:26:00+00,352,7,40.763406,504,1 Ave & E 15 St,40.732219,-73.981656,15173,Subscriber,1983,1
7,NaN,E 4 St & 2 Ave,NaN,77,-73.989780,913,2015-02-01 00:04:00+00,2015-02-01 00:19:00+00,439,8,40.726281,116,W 17 St & 8 Ave,40.741776,-74.001497,17862,Subscriber,1955,1
8,NaN,Washington Pl & Broadway,NaN,78,-73.994046,759,2015-02-01 00:04:00+00,2015-02-01 00:17:00+00,335,9,40.729039,2012,E 27 St & 1 Ave,40.739445,-73.976806,21183,Subscriber,1985,2
9,NaN,Greenwich Ave & 8 Ave,NaN,79,-74.002638,585,2015-02-01 00:05:00+00,2015-02-01 00:15:00+00,284,10,40.739017,444,Broadway & W 24 St,40.742354,-73.989151,14843,Subscriber,1982,1


In [5]:
#TASK 1.2

query = '''
SELECT * FROM fb55.citibike
ORDER BY cartodb_id DESC
Limit 10;  
'''
table = get_data(query)
table

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 4 St & 2 Ave,NaN,46199,-73.989780,917,2015-02-07 23:59:00+00,2015-02-08 00:15:00+00,439,46200,40.726281,417,Barclay St & Church St,40.712912,-74.010202,20998,Subscriber,1965,2
1,NaN,Carmine St & 6 Ave,NaN,46198,-74.002150,548,2015-02-07 23:58:00+00,2015-02-08 00:08:00+00,368,46199,40.730386,334,W 20 St & 7 Ave,40.742388,-73.997262,19540,Subscriber,1983,2
2,NaN,5 Ave & E 29 St,NaN,46197,-73.986831,392,2015-02-07 23:57:00+00,2015-02-08 00:03:00+00,474,46198,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,15545,Subscriber,1986,1
3,NaN,5 Ave & E 29 St,NaN,46196,-73.986831,428,2015-02-07 23:57:00+00,2015-02-08 00:04:00+00,474,46197,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,16395,Subscriber,1986,2
4,NaN,W 20 St & 8 Ave,NaN,46195,-74.000040,689,2015-02-07 23:57:00+00,2015-02-08 00:08:00+00,470,46196,40.743453,325,E 19 St & 3 Ave,40.736245,-73.984738,15585,Subscriber,1953,1
5,NaN,1 Ave & E 44 St,NaN,46194,-73.969053,1422,2015-02-07 23:57:00+00,2015-02-08 00:20:00+00,455,46195,40.750020,265,Stanton St & Chrystie St,40.722293,-73.991475,20184,Subscriber,1960,2
6,NaN,E 20 St & 2 Ave,NaN,46193,-73.982050,993,2015-02-07 23:56:00+00,2015-02-08 00:13:00+00,461,46194,40.735877,295,Pike St & E Broadway,40.714067,-73.992939,16722,Subscriber,1974,1
7,NaN,Warren St & Church St,NaN,46192,-74.009106,1165,2015-02-07 23:56:00+00,2015-02-08 00:15:00+00,152,46193,40.714740,325,E 19 St & 3 Ave,40.736245,-73.984738,16978,Subscriber,1959,1
8,NaN,W 47 St & 10 Ave,NaN,46191,-73.993012,1248,2015-02-07 23:56:00+00,2015-02-08 00:17:00+00,495,46192,40.762699,432,E 7 St & Avenue A,40.726218,-73.983799,16300,Subscriber,1984,1
9,NaN,W 17 St & 8 Ave,NaN,46190,-74.001497,306,2015-02-07 23:55:00+00,2015-02-08 00:01:00+00,116,46191,40.741776,494,W 26 St & 8 Ave,40.747348,-73.997236,14736,Subscriber,1983,2


In [6]:
#Task 1.3
#List all unique start_station_id values

query = '''
SELECT DISTINCT start_station_id FROM fb55.citibike
'''
table = get_data(query)
table.head()

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN


In [7]:
#Task 1.4: Aggregation functions
#Count the number of trips (aka wc -l in SQL)
#Find the average/min/max trip duration

query = '''
SELECT COUNT(cartodb_id) FROM fb55.citibike
'''
table = get_data(query)
table.head()

,count,Unnamed: 1
0,46200,NaN


In [8]:
query = '''
SELECT AVG(tripduration) FROM fb55.citibike
'''
table = get_data(query)
table

,avg,Unnamed: 1
0,675.865823,NaN


In [9]:
query = '''
SELECT MAX(tripduration) FROM fb55.citibike
'''
table = get_data(query)
table

,max,Unnamed: 1
0,43016,NaN


In [10]:
query = '''
SELECT MIN(tripduration) FROM fb55.citibike
'''
table = get_data(query)
table

,min,Unnamed: 1
0,60,NaN


# Task 2 — Working with date/time

2.1 Selecting trips started on Feb-02-2015 only

2.2 Selecting trips started on the weekends

- What are average trip duration during weekends?

2.3 Can we do the same for weekday?

In [11]:
#Task 2.1

query = '''
SELECT * FROM fb55.citibike WHERE
EXTRACT (YEAR FROM starttime) = 2015 AND
EXTRACT (MONTH FROM starttime) = 2 AND
EXTRACT (DAY FROM starttime) = 2;
'''
table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 5 St & Avenue C,NaN,11,-73.979955,1312,2015-02-02 11:07:00+00,2015-02-02 11:29:00+00,393,6920,40.722992,476,E 31 St & 3 Ave,40.743943,-73.979661,17720,Subscriber,1955.0,1
3,NaN,9 Ave & W 16 St,NaN,627,-74.004432,444,2015-02-02 00:40:00+00,2015-02-02 00:48:00+00,463,6477,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20233,Subscriber,1971.0,1
4,NaN,9 Ave & W 16 St,NaN,991,-74.004432,438,2015-02-02 00:40:00+00,2015-02-02 00:47:00+00,463,6478,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20264,Subscriber,1981.0,2


In [12]:
#Task 2.2 Selecting trips on weekends

query = '''
SELECT * FROM fb55.citibike WHERE
EXTRACT (DOW FROM starttime) IN (5,6)
'''
table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 17 St & Broadway,NaN,31009,-73.990093,409,2015-02-06 05:41:00+00,2015-02-06 05:48:00+00,497,30964,40.737050,379,W 31 St & 7 Ave,40.749156,-73.991600,17096,Subscriber,1979.0,2
1,NaN,E 51 St & 1 Ave,NaN,32290,-73.965930,561,2015-02-06 08:39:00+00,2015-02-06 08:49:00+00,454,32252,40.754557,167,E 39 St & 3 Ave,40.748901,-73.976049,17997,Subscriber,1981.0,1
2,NaN,E 47 St & Park Ave,NaN,32672,-73.974987,448,2015-02-06 09:04:00+00,2015-02-06 09:12:00+00,359,32635,40.755103,477,W 41 St & 8 Ave,40.756405,-73.990026,16533,Subscriber,1968.0,1
3,NaN,E 40 St & 5 Ave,NaN,33354,-73.981632,497,2015-02-06 10:13:00+00,2015-02-06 10:22:00+00,153,33320,40.752062,518,E 39 St & 2 Ave,40.747804,-73.973442,16009,Subscriber,1974.0,1
4,NaN,E 15 St & 3 Ave,NaN,33790,-73.986923,932,2015-02-06 11:38:00+00,2015-02-06 11:54:00+00,297,33756,40.734232,297,E 15 St & 3 Ave,40.734232,-73.986923,17969,Subscriber,1964.0,2


In [13]:
# Average trip duration during weekends

query = '''
SELECT AVG(tripduration) FROM fb55.citibike WHERE
EXTRACT (DOW FROM starttime) IN (0,6)
'''
table = get_data(query)
table.head()

,avg,Unnamed: 1
0,662.942181,NaN


In [14]:
#Task 2.3 Selecting trips on weekdays

query = '''
SELECT * FROM fb55.citibike WHERE
EXTRACT (DOW FROM starttime) IN (1,2,3,4,5)
'''
table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 39 St & 3 Ave,NaN,8327,-73.976049,441,2015-02-03 08:02:00+00,2015-02-03 08:10:00+00,167,8263,40.748901,526,E 33 St & 5 Ave,40.747659,-73.984907,21518,Subscriber,1975.0,1
3,NaN,10 Ave & W 28 St,NaN,9435,-74.001768,1475,2015-02-03 12:31:00+00,2015-02-03 12:56:00+00,489,9375,40.750664,455,1 Ave & E 44 St,40.750020,-73.969053,19863,Subscriber,1984.0,2
4,NaN,E 2 St & 2 Ave,NaN,10472,-73.990697,420,2015-02-03 16:45:00+00,2015-02-03 16:52:00+00,403,10411,40.725029,297,E 15 St & 3 Ave,40.734232,-73.986923,19257,Subscriber,1984.0,1


In [15]:
# Average trip duration on weekdays

query = '''
SELECT AVG(tripduration) FROM fb55.citibike WHERE
EXTRACT (DOW FROM starttime) IN (1,2,3,4,5)
'''
table = get_data(query)
table.head()

,avg,Unnamed: 1
0,681.052292,NaN


# Task 3

Task 3 — Working with Space

3.1: Showing the list of start station locations

- Using GROUP BY

3.2: Showing the number of trips started per station … but only for stations within 500m of Time Square!

- The coordinates of Time Square is (40.7577,-73.9857)

In [16]:
# task 3.1 list of start station locations

query = '''
SELECT DISTINCT start_station_name, COUNT(tripduration) FROM fb55.citibike
GROUP BY start_station_name; 
'''
table = get_data(query)
table

,start_station_name,count
0,10 Ave & W 28 St,203
1,11 Ave & W 27 St,172
2,11 Ave & W 41 St,229
3,11 Ave & W 59 St,120
4,12 Ave & W 40 St,91
5,1 Ave & E 15 St,397
6,1 Ave & E 44 St,110
7,2 Ave & E 31 St,302
8,2 Ave & E 58 St,64
9,3 Ave & Schermerhorn St,19


In [17]:
# task 3.2 number of trips started per station within 500m of Times Square
# The coordinates of Time Square is (40.7577,-73.9857)
# code credit: http://daynebatten.com/2015/09/latitude-longitude-distance-sql/

query = '''
SELECT DISTINCT start_station_name, COUNT(tripduration) FROM fb55.citibike
WHERE 2 * 3961 * asin(sqrt((sin(radians((start_station_latitude - 40.7577) / 2))) ^ 2 + cos(radians(start_station_latitude)) * cos(radians(40.7577)) * (sin(radians((start_station_longitude - -73.9857) / 2))) ^ 2)) <= 0.310686
GROUP BY start_station_name
; 
'''
table = get_data(query)
table

,start_station_name,count
0,Broadway & W 41 St,251
1,Broadway & W 49 St,213
2,W 41 St & 8 Ave,507
3,W 42 St & 8 Ave,221
4,W 43 St & 6 Ave,112
5,W 45 St & 6 Ave,141
6,W 45 St & 8 Ave,141


# Task 4 — Putting it all together
4.1: Find the station that had the longest average trip duration during:
weekends and within 500m of TimeSquare!

4.2: Extra Credit: create lines for trips started from stations within 500m of Times
Squares and lasted less than 2 hours. The number of trips per each
pair of stations are output as attributes of these lines. 

In [18]:
# Task 4.1 : longest avg trip duration, weekends, 500m of timesquare
# covert 500m to 0.310686 miles

query = '''
SELECT DISTINCT start_station_name, AVG(tripduration) FROM fb55.citibike WHERE
EXTRACT (DOW FROM starttime) IN (0,6) AND
2 * 3961 * asin(sqrt((sin(radians((start_station_latitude - 40.7577) / 2))) ^ 2 + cos(radians(start_station_latitude)) * cos(radians(40.7577)) * (sin(radians((start_station_longitude - -73.9857) / 2))) ^ 2)) <= 0.310686
GROUP BY start_station_name
ORDER BY AVG(tripduration) DESC
LIMIT 1
; 
'''
table = get_data(query)
table

,start_station_name,avg
0,Broadway & W 49 St,1010.104167


# Extra Credit

In [19]:
# Task 4.2 for
# trips taken within 500m of Times Square
# trips less than 2 hours

query = '''
SELECT start_station_id, end_station_id, COUNT(tripduration) FROM fb55.citibike WHERE
tripduration < 7200 AND
2 * 3961 * asin(sqrt((sin(radians((start_station_latitude - 40.7577) / 2))) ^ 2 + cos(radians(start_station_latitude)) * cos(radians(40.7577)) * (sin(radians((start_station_longitude - -73.9857) / 2))) ^ 2)) <= 0.310686
GROUP BY start_station_id , end_station_id
ORDER BY COUNT(tripduration) DESC
; 
'''
table = get_data(query)
table

,start_station_id,end_station_id,count
0,477,318,31
1,477,359,21
2,529,478,17
3,477,517,17
4,173,318,15
5,477,519,15
6,524,318,14
7,493,492,14
8,477,153,14
9,465,492,14
